In [151]:
import pandas as pd

In [152]:
df=pd.read_csv("pre_ctrain.csv")

In [153]:
# Separate features and target
X = df.drop('label', axis=1)
y = df['label']

In [154]:
df_test=pd.read_csv("pre_ctest.csv")

In [155]:
# Separate features and target
X_test = df_test.drop('label', axis=1)
y_test = df_test['label']

In [156]:
df_val=pd.read_csv("pre_cval.csv")

In [157]:
# Separate features and target
X_val = df_val.drop('label', axis=1)
y_val = df_val['label']

In [158]:
df.shape

(76330, 79)

In [159]:
df_test.shape

(16357, 79)

In [160]:
df_val.shape

(16357, 79)

In [161]:
from sklearn.preprocessing import StandardScaler

In [162]:
# Initialize the scaler
scaler = StandardScaler()

In [163]:
# Fit on training data and transform
X = scaler.fit_transform(X)

In [164]:
# Transform test data
X_test = scaler.transform(X_test)

In [165]:
# Transform validation data
X_val = scaler.transform(X_val)

In [166]:
scalerb=scaler

In [167]:
import joblib
joblib.dump(scaler,"scaler_alc.pkl")

['scaler_alc.pkl']

In [168]:
print(df.columns.to_list())

[' Destination Port', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count', ' ACK Flag Count', ' URG Flag 

In [169]:
old_features_alc=df.columns.to_list()

In [170]:
old_features_alc.remove('label')

In [171]:
joblib.dump(old_features_alc,"old_features_alc.pkl")

['old_features_alc.pkl']

In [172]:
from sklearn.feature_selection import VarianceThreshold, SelectKBest, mutual_info_classif


In [173]:
X = pd.DataFrame(X, columns=old_features_alc)

In [174]:
X_val = pd.DataFrame(X_val, columns=old_features_alc)

In [175]:
X_test = pd.DataFrame(X_test, columns=old_features_alc)

In [176]:
print(old_features_alc)

[' Destination Port', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count', ' ACK Flag Count', ' URG Flag 

In [177]:
leakage_features = [
    ' Flow Duration',
    ' Total Fwd Packets',
    ' Total Backward Packets',
    'Total Length of Fwd Packets',
    ' Total Length of Bwd Packets',
    'Flow Bytes/s',
    ' Flow Packets/s',
    'Fwd IAT Total',
    'Bwd IAT Total',
    'Active Mean',
    ' Active Std',
    ' Active Max',
    ' Active Min',
    'Idle Mean',
    ' Idle Std',
    ' Idle Max',
    ' Idle Min'
]


In [178]:
X_c=X

In [179]:
X = X.drop(columns=[col for col in leakage_features if col in X.columns])

In [180]:
print(X.shape[1])

61


In [181]:
var_thresh = VarianceThreshold(threshold=0.01)
X = X.loc[:, var_thresh.fit(X).get_support()]

In [182]:
a=X.shape[1]

In [183]:
print("features dropped: ",61-a)

features dropped:  8


In [184]:
import numpy as np

In [185]:
corr_matrix = X.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop_corr = [c for c in upper_tri.columns if any(upper_tri[c] > 0.8)]


In [186]:
X_corr = X.drop(columns=to_drop_corr)
print(f"Correlation Filter: Dropped {len(to_drop_corr)}, Remaining {X_corr.shape[1]} features")

Correlation Filter: Dropped 27, Remaining 26 features


In [187]:
X=X_corr

In [188]:
features_alc=X.columns.to_list()

In [189]:
joblib.dump(features_alc,"features_alc_m.pkl")

['features_alc_m.pkl']

In [190]:
#check for dupilicates
print("Train duplicates:", X.duplicated().sum())

Train duplicates: 34


In [191]:

duplicates_mask = X.duplicated(keep='first')


In [192]:
X = X[~duplicates_mask].reset_index(drop=True)


In [193]:
y = y[~duplicates_mask].reset_index(drop=True)


In [194]:
y = y.astype(int)


In [195]:
df_new = X.copy()       # copy X to preserve original
df_new['label'] = y

In [196]:
# Save to CSV
df_new.to_csv("ctrain_f.csv", index=False)

In [197]:
X_test = X_test[features_alc]

In [198]:
X_test.head()

,Destination Port,Fwd Packet Length Max,Fwd Packet Length Min,Bwd Packet Length Max,Bwd Packet Length Min,Flow IAT Mean,Flow IAT Std,Flow IAT Min,Bwd IAT Std,Fwd PSH Flags,...,FIN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,Down/Up Ratio,Subflow Fwd Packets,Subflow Fwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward
0,-0.446535,1.627812,-0.317599,-0.306476,-0.604146,0.367010,0.197303,-0.056144,0.524930,-0.219449,...,-0.177444,-0.016589,1.494444,-0.657391,-0.327430,-1.066735,0.004436,0.475163,1.529584,-0.226699
1,2.605095,-0.297057,-0.317599,-0.460550,-0.604146,-0.295119,-0.371791,-0.056129,-0.242941,-0.219449,...,-0.177444,-0.016589,-0.669145,1.521164,3.054083,0.452736,-0.012080,-0.113690,-0.490487,-0.202741
2,-0.447990,-0.252039,0.234907,-0.425189,0.401278,-0.291426,-0.368206,-0.056155,-0.242941,-0.219449,...,-0.177444,-0.016589,-0.669145,-0.657391,-0.327430,0.452736,-0.010428,-0.099830,-0.498474,-0.237568
3,-0.447990,-0.250675,0.251650,-0.353961,2.426488,-0.295119,-0.371778,-0.056156,-0.242941,-0.219449,...,-0.177444,-0.016589,-0.669145,-0.657391,-0.327430,0.452736,-0.010428,-0.099410,-0.498474,-0.237568
4,-0.446535,-0.288872,-0.217143,-0.460550,-0.604146,-0.295119,-0.371791,-0.056128,-0.242941,-0.219449,...,-0.177444,-0.016589,-0.669145,1.521164,-0.327430,-1.066735,-0.010428,-0.111170,-0.480972,-0.237568


In [199]:
X_test.head

<bound method NDFrame.head of         Destination Port   Fwd Packet Length Max   Fwd Packet Length Min  \
0              -0.446535                1.627812               -0.317599   
1               2.605095               -0.297057               -0.317599   
2              -0.447990               -0.252039                0.234907   
3              -0.447990               -0.250675                0.251650   
4              -0.446535               -0.288872               -0.217143   
...                  ...                     ...                     ...   
16352          -0.447990               -0.265681                0.067481   
16353          -0.447990               -0.257496                0.167937   
16354          -0.447990               -0.246582                0.301877   
16355          -0.447990               -0.228848                0.519531   
16356          -0.446535               -0.288872               -0.217143   

       Bwd Packet Length Max   Bwd Packet Length Min   Fl

In [200]:
X_val = X_val[features_alc]

In [201]:
#check for dupilicates
print("Test duplicates:", X_test.duplicated().sum())

Test duplicates: 3


In [202]:
#check for dupilicates
print("Validation duplicates:", X_val.duplicated().sum())

Validation duplicates: 2


In [203]:

duplicates_mask = X_val.duplicated(keep='first')


In [204]:
X_val = X_val[~duplicates_mask].reset_index(drop=True)
y_val = y_val[~duplicates_mask].reset_index(drop=True)

In [205]:
y_val = y_val.astype(int)


In [206]:
df_new = X_val.copy()       # copy X to preserve original
df_new['label'] = y_val

In [207]:
# Save to CSV
df_new.to_csv("cval.csv", index=False)

In [208]:
duplicates_mask = X_test.duplicated(keep='first')


In [209]:
X_test = X_test[~duplicates_mask].reset_index(drop=True)
y_test = y_test[~duplicates_mask].reset_index(drop=True)

In [210]:
y_test = y_test.astype(int)


In [211]:
df_new = X_test.copy()       # copy X to preserve original
df_new['label'] = y_test

In [212]:
# Save to CSV
df_new.to_csv("ctest.csv", index=False)